In [117]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur

In [118]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect = augurApp.ghtorrentplus()
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            'derek', 'picklehead5000', 'ghdata.sociallycompute.io',\
            '3306', 'chaoss'
        )
db = s.create_engine(DB_STR)

In [119]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
token = 'insert_token'
own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'

# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning
repo = GitHub(owner=own,repository=repo_url,api_token=token)

In [126]:
#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
k=0
#for issues in repo.fetch(from_date=date):
columns1 = "backend_name","owner","repo","subject",\
          "status","category","issue_number","user","body"
df = pd.DataFrame(columns=columns1)
df.to_sql(name="github_issues", con=db,\
          if_exists='replace',index=False,
           dtype={'backend_name': s.types.VARCHAR(length=300),
                  'owner': s.types.VARCHAR(length=300),
                  'repo': s.types.VARCHAR(length=300),
                  'subject': s.types.VARCHAR(length=300),
                  'status': s.types.VARCHAR(length=10),
                  'category': s.types.VARCHAR(length=10),
                  'issue_number': s.types.Integer,
                  'user': s.types.VARCHAR(length=100),
                  'body':s.types.VARCHAR(length=12000)               
           })
for issues in repo.fetch():
    # Create the object (dictionary) to upload to ElasticSearch
    # Create the object (dictionary) to upload to ElasticSearch
    #if(i==90):
    if 'pull_request' in issues['data']:
        #print(issues['data']['number'])
        continue
    else:
        #if(issues['data']['state'] == 'closed' and issues['data']\
        #   ['number'] == 165): 
        text = issues['data']['body']
        user = issues['data']['user']['login']
        num = issues['data']['number']
        store = [issues['backend_name'],own,repo_url,issues['data']['title'],\
        issues['data']['state'],issues['category'],num,user,text]
        #print(store[:])
        #print(store[6])
        print("Issue number: ",issues['data']['number'])
        #if(k > 1):
            #k+=1
            #continue
        '''print("Backend_name: ", issues['backend_name'])
        print("Category: ", issues['category'])
        print("Subject: ", issues['data']['title'])
        print("State: ", issues['data']['state'])
        print("First body:",issues['data']['body'])
        print("User: ", issues['data']['user']['login'])
        print("\n\n\n")'''
        df_user = pd.DataFrame([store],columns=columns1)
        df = df.append(df_user)
        for i in range(len(issues['data']['comments_data'])):
            temp=0
            store[6] = issues['data']['number']
            store[7] = issues['data']['comments_data'][i]\
            ['user_data']['login']
            store[8] = issues['data']['comments_data'][i]['body']
            df_user = pd.DataFrame([store],columns=columns1)
            df = df.append(df_user)
            #print(issues['data']['comments_data'][i]\
            #      ['user_data']['login'])
            #print(issues['data']['comments_data'][i]['body'])
          
    if(df.shape[0] > 1000):
        df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
        df = pd.DataFrame(columns=columns1)
        print("Broken")
        #break
    #print(issues['data'][''])
    print(k)
    k+=1
    #if(k == 9):
    #    break
print(df)
if(df.shape[0] < 1000):
    df.to_sql(name='github_issues', con=db,\
                  if_exists='append',index=False)
    df = pd.DataFrame(columns=columns1)
    print("Broken")
print("Total Number of issues: ", k)
print(df.shape[0])

Issue number:  1
0
Issue number:  14
1
Issue number:  3
2
Issue number:  4
3
Issue number:  5
4
Issue number:  8
5
Issue number:  10
6
Issue number:  17
7
Issue number:  6
8
Issue number:  9
9
Issue number:  15
10
Issue number:  13
11
Issue number:  33
12
Issue number:  20
13
Issue number:  35
14
Issue number:  21
15
Issue number:  36
16
Issue number:  29
17
Issue number:  25
18
Issue number:  32
19
Issue number:  39
20
Issue number:  43
21
Issue number:  50
22
Issue number:  46
23
Issue number:  51
24
Issue number:  7
25
Issue number:  16
26
Issue number:  49
27
Issue number:  54
28
Issue number:  41
29
Issue number:  59
30
Issue number:  61
31
Issue number:  63
32
Issue number:  22
33
Issue number:  65
34
Issue number:  68
35
Issue number:  70
36
Issue number:  71
37
Issue number:  40
38
Issue number:  44
39
Issue number:  74
40
Issue number:  77
41
Issue number:  62
42
Issue number:  79
43
Issue number:  82
44
Issue number:  85
45
Issue number:  91
46
Issue number:  90
47
Issue numb

/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x91\\x8D o...' for column 'body' at row 65")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x89 \\x0A...' for column 'body' at row 107")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x8E\\x89  ...' for column 'body' at row 201")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x91\\x8D ' for column 'body' at row 212")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x9

Broken
68
Issue number:  120
69
Issue number:  127
70
Issue number:  67
71
Issue number:  97
72
Issue number:  130
73
Issue number:  92
74
Issue number:  138
75
Issue number:  128
76
Issue number:  129
77
Issue number:  116
78
Issue number:  135
79
Issue number:  109
80
Issue number:  142
81
Issue number:  102
82
Issue number:  143
83
Issue number:  148
84
Issue number:  153
85
Issue number:  150
86
Issue number:  145
87
Issue number:  146
88
Issue number:  140
89
Issue number:  154
90
Issue number:  156
91
Issue number:  155
92
Issue number:  160
93
Issue number:  164
94
Issue number:  162
95
Issue number:  170
96
Issue number:  123
97
Issue number:  114
98
Issue number:  147
99
Issue number:  24
100
Issue number:  42
101
Issue number:  55
102
Issue number:  48
103
Issue number:  175
104
Issue number:  104
105
Issue number:  2
106
Issue number:  118
107
Issue number:  124
108
Issue number:  119
109
Issue number:  110
110
Issue number:  96
111
Issue number:  126
112
Issue number:  139


/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x8E\\x89)' for column 'body' at row 4")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x91\\x8D S...' for column 'body' at row 88")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x91\\x8D s...' for column 'body' at row 158")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1265, "Data truncated for column 'body' at row 247")
  result = self._query(query)
/home/keanu/anaconda3/envs/augur/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xF0\\x9F\\x98\\x83 ' for column 'body' at row 279")
  r

Broken
Total Number of issues:  126
0
